In [1]:
import torch
import llm_transparency_tool.routes.graph
from llm_transparency_tool.models.tlens_model import TransformerLensTransparentLlm
from llm_transparency_tool.models.transparent_llm import TransparentLlm

from llm_transparency_tool.models.tlens_model import (
TransformerLensTransparentLlm,
)
model = TransformerLensTransparentLlm("meta-llama/Llama-3.2-1B")


/Users/harshitgupta/miniforge3/envs/llmtt2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model.run("John is a good")

graph = llm_transparency_tool.routes.graph.build_full_graph(
    model,0,0.4
)

2025-03-27 16:18:46.548 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-27 16:18:46.598 
  command:

    streamlit run /Users/harshitgupta/miniforge3/envs/llmtt2/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-03-27 16:18:46.598 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-27 16:18:46.598 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-27 16:18:47.104 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-27 16:18:47.104 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-27 16:19:00.714 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-27 16:19:00.71

Loaded pretrained model meta-llama/Llama-3.2-1B into HookedTransformer


In [3]:
print(graph)
print(graph.edge_subgraph)
print(type(graph))

DiGraph with 245 nodes and 640 edges
<bound method Graph.edge_subgraph of <networkx.classes.digraph.DiGraph object at 0x1537fea50>>
<class 'networkx.classes.digraph.DiGraph'>
